# Rozpoznawanie ras psów z wykorzystaniem biblioteki Keras
### Michał Foryt, Szczepan Gabiec, Paweł Wróblewski

<img src="meme2.jpg" style="width: 400px;">

Celem projektu jest stworzenie algorytmu służącego do rozpoznawaniu ras psów porzy wykorzystaniu biblioteki Keras, w oparciu o TensorFlow. Zbiór z którego korzystaliśmy pochodzi z jednego z konkursów zorganizowanych przez portal Kaggle i znajduje się pod tym linkiem: https://www.kaggle.com/c/dog-breed-identification. Z uwagi na jego rozmiar (~750MB) nie bedzie on dołączony bezpośrednio do kodu.  

W skład zbioru wchodzi około 10 tysięcy zdjęć, a wśród nich można wyróźnić 120 klas (ras psów). Pracując na tych danych, będziemy mogli stworzyć algorytm bazujący na głębokim uczeniu sieci neuronowych. W dużym uproszczeniu, zadaniem tego typu algorytmu jest rozpoznawanie obrazów podobnie, jak robi to umysł ludzki - "oglądając" ogromną pulę zdjęć z czasem rozpoznaje charakterystyczne cechy dla danego obiektu, przykładowo że rottweilera można poznać po tym, że jest duży i ma krótką, ciemną sierść, a york ma krótkie łapy i jest mocno owłosiony. Innymi słowy, algorytm najpierw zbiera dużą ilość danych, a następnie pozwala komputerowi zapoznać się z każdym z nich. Opierając się na dużych bazach danych i zauważając pojawiające się wzorce, komputery mogą rozpoznać obrazy i sformułować odpowiednie tagi i kategorie.

Formalnie można nakreślić następujacy przebieg metody:
- Na wejściu algorytm otrzymuje zbiór zawierający N obrazów, każdy z nich przypisany jest do jednej z K klas. 
- Następnie wykorzystuje się zbiór treningowy do szkolenia klasyfikatora tak, aby był on w stanie jak najepiej przyporządkować etykietę do zdjęcia.
- Na koniec ocenia się jakość klasyfikatora, prosząc go o przewidywanie etykiet dla nowego zestawu obrazów, których nigdy wcześniej nie widział, po czym porównamy prawdziwe etykiety tych obrazów z przewidywanymi przez klasyfikator.


## Konwolucyjne Sieci Neuronowe 
Konwolucyjne Sieci Neuronowe (ang. <i>CNN, Convolutional neural networks</i>, tłumaczone także jako <i>Splotowe Sieci Neuronowe</i>) w sprytny sposób redukują liczbę przyjmowanych parametrów. Zamiast działać na sieci, w której neurony są połączone każdy z każdym, podejście jakie prezentują CNN wykorzystują wielokrotnie te same parametry. Kluczem do sukcesu konwolucyjnych sieci jest fakt, że wychodza one z założenia, że wystarczy lokalne zrozumienie obrazu. Innymi słowy, algorytm skupia się na tym, by  stopniowo filtrować różne części danych uczących i wyostrzać ważne cechy w procesie dyskryminacji wykorzystanym do rozpoznawania lub klasyfikacji wzorców. W praktyce zaletą takiego podejścia jest posiadanie mniejszej liczby parametrów, co przekłada się na znaczne zmniejszenie czasu potrzebnego do wytrenowania modelu.

Rozważmy obraz o wymiarze 256 × 256 pikseli. Zamiast przetwarzać cały obraz naraz, CNN może skutecznie skanować go po kawałku - powiedzmy, patrząc na fragment o wymiarach 5 × 5. Taka ramka o wymiarach 5 × 5px przesuwa się wzdłuż obrazu (zwykle od lewej do prawej i od góry do dołu), jak pokazano na poniższym rysunku. 

<img src="cnn_concept.jpeg" style="width: 350px;">

Tempo przesuwania się takiej ramki nazywamy "długością kroku". Na przykład długość kroku 2 oznacza, że okno 5 × 5 przesuwa się o 2 piksele na raz, aż obejmie cały obraz. Taka ramka 5 x 5 pikseli przekłada się na macierz wag o wymiarze 5 x 5. 

Tego typu operacja ma miejsce w warstwie konwolucyjnej sieci neuronowej. Typowa CNN posiada wiele tego typu warste. Każda z nich zazwyczaj generuje wiele różnych splotów (ang. <i>convolutions</i>). Co za tym idzie, macierz wagowa takiego tensora (czyli obiektu matematycznego będącego uogólnieniem pojęcia wektora) ma wymiary 5 × 5 x n, gdzie n liczbą konwolucji (splotów).

Przykładowo, załóżmy że przepuszczamy rozważany obraz przez pojedynczą warstwę splotu jako macierz wagowa 5 x 5 x 64 z ramką 5 x 5. Co za ty idzie, taki model posiada 5 x 5 x64 = 1600 parametrów, podczas gdy pełna sieć dla obrazu 256 x 256px wymagałaby zastosowania 65 536 parametrów.

## Obróbka zbioru danych
Jak już wspomnielismy na początku, wykorzystamy zbiór danych dotyczący rozpoznawania ras psów, który wstępnie został już podzielony na zbiór treningowy i testowy. Nazwa każdego z obrazów jest też jednocześnie jego unikalnym id. Cały zestaw danych zawiera zdjęcia 120 ras, jednak dla urposzczenia modelu przyjmiemy założenie, że ograniczamy się jedynie do rozpoznawania 8 najpopularniejszych.

Dla porządku, wszystkie niezbędne importy zastosujemy poniżej. Dzięki temu uzyskamy większa czytelność oraz podcas tworzenia naszej funkcjonalności będziemy mieli pewność, że wszystkie niezbędne paczki zostały już ściągnięte.  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import matplotlib.pyplot as plt
from shutil import copyfile
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import cv2
import pandas as pd

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

Czytamy dane z pliku labels, które mapują id zdjęcia z jego opisem. na zbiorze treningowym. 

In [3]:
labels = pd.read_csv('labels.csv')
labels_dict = {i:j for i,j in zip(labels['id'],labels['breed'])}
classes = set(labels_dict.values())
images = [f for f in os.listdir('train')]

Tworzymy nowe katalogi, które zawierają zbiory zdjęć - treningowy i testowy. 

In [4]:
if not os.path.exists('training_images'):
        os.makedirs('training_images')

if not os.path.exists('validation_images'):
    os.makedirs('validation_images')

In [6]:
print(classes)

{'sussex_spaniel', 'french_bulldog', 'entlebucher', 'bloodhound', 'miniature_pinscher', 'malamute', 'dandie_dinmont', 'soft-coated_wheaten_terrier', 'norwich_terrier', 'basset', 'ibizan_hound', 'beagle', 'boston_bull', 'silky_terrier', 'whippet', 'labrador_retriever', 'toy_poodle', 'doberman', 'brabancon_griffon', 'samoyed', 'toy_terrier', 'rottweiler', 'lhasa', 'dhole', 'pekinese', 'kelpie', 'saint_bernard', 'standard_poodle', 'norfolk_terrier', 'otterhound', 'miniature_poodle', 'airedale', 'chow', 'pomeranian', 'shih-tzu', 'lakeland_terrier', 'pug', 'american_staffordshire_terrier', 'vizsla', 'miniature_schnauzer', 'irish_wolfhound', 'sealyham_terrier', 'irish_terrier', 'german_short-haired_pointer', 'groenendael', 'dingo', 'shetland_sheepdog', 'briard', 'bernese_mountain_dog', 'english_springer', 'basenji', 'chihuahua', 'cardigan', 'staffordshire_bullterrier', 'border_terrier', 'flat-coated_retriever', 'boxer', 'bull_mastiff', 'bedlington_terrier', 'pembroke', 'keeshond', 'mexican_h

Otwieramy folder z danymi treningowymi. Dla każdego z nich przygotowujemy katalogi, do których następnie będziemy zapisywać zdjęcia ras w osobne podkatalogi.

In [5]:
os.chdir('training_images')
for curClass in classes:    
    if not os.path.exists(curClass):
        os.makedirs(curClass)

 To samo robimy dla zbioru testowego. 

In [ ]:
os.chdir('../validation_images')
for curClass in classes:    
    if not os.path.exists(curClass):
        os.makedirs(curClass)

Wracamy do głównego folderu. Po przygotowaniu odpowiednich katalogów możemy je wypełnić. Mamy około 20.5 tysięcy zdjęć, dzielimy je w proporcjach około 40:60 między zbiór treningowy i testowy - pierwsze 8 tysięcy zdjęć kopiujemy do odpowiednich katalogów w   

In [7]:
os.chdir('..')
count = 0 
destination_directory = 'training_images/'
for item in images:
    if count >7999:
        destination_directory = 'validation_images/'
    filekey = os.path.splitext(item)[0]
    target_file = destination_directory+labels_dict[filekey]+'/'+item
    if not os.path.exists(target_file):
        copyfile('train/'+item, target_file)
    print(labels_dict[filekey])
    count +=1

chow
french_bulldog
redbone
chesapeake_bay_retriever
scottish_deerhound
redbone
basset
lakeland_terrier
shetland_sheepdog
cardigan
bluetick
tibetan_terrier
mexican_hairless
bedlington_terrier
groenendael
italian_greyhound
golden_retriever
italian_greyhound
komondor
lhasa
borzoi
sealyham_terrier
whippet
african_hunting_dog
samoyed
miniature_pinscher
pekinese
german_short-haired_pointer
afghan_hound
kuvasz
eskimo_dog
rottweiler
basenji
african_hunting_dog
airedale
australian_terrier
maltese_dog
great_pyrenees
greater_swiss_mountain_dog
toy_terrier
pug
bluetick
samoyed
italian_greyhound
samoyed
shih-tzu
miniature_pinscher
bouvier_des_flandres
afghan_hound
soft-coated_wheaten_terrier
newfoundland
wire-haired_fox_terrier
scotch_terrier
shih-tzu
lakeland_terrier
basenji
rottweiler
shih-tzu
chow
briard
basenji
whippet
japanese_spaniel
kuvasz
clumber
walker_hound
bernese_mountain_dog
bernese_mountain_dog
bernese_mountain_dog
irish_wolfhound
keeshond
kerry_blue_terrier
basenji
irish_water_spani

bedlington_terrier
otterhound
entlebucher
papillon
norfolk_terrier
pomeranian
scotch_terrier
miniature_schnauzer
bloodhound
airedale
french_bulldog
bedlington_terrier
pomeranian
scottish_deerhound
siberian_husky
brabancon_griffon
vizsla
blenheim_spaniel
clumber
welsh_springer_spaniel
scottish_deerhound
lhasa
toy_poodle
afghan_hound
bernese_mountain_dog
labrador_retriever
german_short-haired_pointer
komondor
brittany_spaniel
great_pyrenees
pekinese
cardigan
toy_terrier
mexican_hairless
doberman
toy_poodle
tibetan_terrier
pembroke
beagle
west_highland_white_terrier
yorkshire_terrier
irish_water_spaniel
entlebucher
miniature_pinscher
lakeland_terrier
cocker_spaniel
lakeland_terrier
chihuahua
bernese_mountain_dog
miniature_poodle
appenzeller
basset
chow
cocker_spaniel
black-and-tan_coonhound
maltese_dog
greater_swiss_mountain_dog
gordon_setter
bernese_mountain_dog
tibetan_mastiff
lakeland_terrier
beagle
american_staffordshire_terrier
scottish_deerhound
old_english_sheepdog
wire-haired_fox_

appenzeller
ibizan_hound
chihuahua
soft-coated_wheaten_terrier
irish_wolfhound
italian_greyhound
weimaraner
pomeranian
border_terrier
golden_retriever
kelpie
irish_setter
french_bulldog
australian_terrier
siberian_husky
siberian_husky
blenheim_spaniel
standard_poodle
scotch_terrier
staffordshire_bullterrier
borzoi
dandie_dinmont
sealyham_terrier
clumber
bouvier_des_flandres
bernese_mountain_dog
gordon_setter
kerry_blue_terrier
chihuahua
flat-coated_retriever
great_dane
labrador_retriever
irish_setter
gordon_setter
cocker_spaniel
beagle
airedale
greater_swiss_mountain_dog
cairn
norfolk_terrier
boxer
lakeland_terrier
doberman
greater_swiss_mountain_dog
leonberg
scottish_deerhound
saluki
pembroke
groenendael
kelpie
chow
bedlington_terrier
scotch_terrier
pug
weimaraner
siberian_husky
sussex_spaniel
miniature_poodle
weimaraner
wire-haired_fox_terrier
irish_terrier
scottish_deerhound
samoyed
italian_greyhound
chow
leonberg
shih-tzu
toy_poodle
great_pyrenees
affenpinscher
tibetan_terrier
blue

newfoundland
groenendael
german_shepherd
affenpinscher
cocker_spaniel
doberman
eskimo_dog
chesapeake_bay_retriever
italian_greyhound
leonberg
rottweiler
border_terrier
lhasa
tibetan_mastiff
norfolk_terrier
scottish_deerhound
papillon
kelpie
entlebucher
giant_schnauzer
clumber
mexican_hairless
norwich_terrier
redbone
soft-coated_wheaten_terrier
sussex_spaniel
sealyham_terrier
entlebucher
norwich_terrier
border_terrier
chow
doberman
bernese_mountain_dog
kuvasz
japanese_spaniel
irish_terrier
greater_swiss_mountain_dog
irish_setter
rhodesian_ridgeback
samoyed
golden_retriever
pomeranian
dandie_dinmont
brittany_spaniel
australian_terrier
african_hunting_dog
sealyham_terrier
english_setter
cairn
giant_schnauzer
rhodesian_ridgeback
dhole
african_hunting_dog
leonberg
clumber
miniature_pinscher
airedale
affenpinscher
scottish_deerhound
chihuahua
irish_wolfhound
vizsla
kerry_blue_terrier
labrador_retriever
ibizan_hound
welsh_springer_spaniel
english_setter
welsh_springer_spaniel
cocker_spaniel
g

old_english_sheepdog
pekinese
miniature_poodle
leonberg
old_english_sheepdog
pomeranian
irish_setter
german_short-haired_pointer
samoyed
bull_mastiff
american_staffordshire_terrier
borzoi
dhole
whippet
silky_terrier
eskimo_dog
shih-tzu
saint_bernard
pomeranian
dandie_dinmont
african_hunting_dog
komondor
cairn
dhole
golden_retriever
shih-tzu
tibetan_mastiff
vizsla
papillon
kelpie
newfoundland
sealyham_terrier
rhodesian_ridgeback
dingo
sealyham_terrier
saluki
dingo
afghan_hound
pembroke
lhasa
afghan_hound
bouvier_des_flandres
maltese_dog
affenpinscher
newfoundland
border_collie
samoyed
kerry_blue_terrier
kerry_blue_terrier
lakeland_terrier
silky_terrier
chihuahua
bernese_mountain_dog
australian_terrier
briard
appenzeller
kelpie
samoyed
irish_setter
malinois
west_highland_white_terrier
curly-coated_retriever
newfoundland
ibizan_hound
scotch_terrier
irish_wolfhound
australian_terrier
irish_wolfhound
dandie_dinmont
kerry_blue_terrier
irish_wolfhound
saint_bernard
basenji
beagle
old_english_

italian_greyhound
welsh_springer_spaniel
clumber
collie
doberman
basenji
african_hunting_dog
labrador_retriever
maltese_dog
west_highland_white_terrier
maltese_dog
basset
irish_wolfhound
irish_setter
boston_bull
redbone
irish_terrier
irish_wolfhound
doberman
old_english_sheepdog
sussex_spaniel
schipperke
airedale
bull_mastiff
collie
entlebucher
cocker_spaniel
affenpinscher
soft-coated_wheaten_terrier
golden_retriever
pembroke
malamute
english_setter
soft-coated_wheaten_terrier
bedlington_terrier
shetland_sheepdog
border_terrier
gordon_setter
brittany_spaniel
scottish_deerhound
tibetan_terrier
giant_schnauzer
groenendael
english_springer
norfolk_terrier
basset
tibetan_mastiff
malamute
clumber
schipperke
great_pyrenees
irish_wolfhound
standard_schnauzer
german_short-haired_pointer
basenji
eskimo_dog
greater_swiss_mountain_dog
irish_terrier
bloodhound
saint_bernard
soft-coated_wheaten_terrier
malinois
collie
basset
flat-coated_retriever
irish_setter
greater_swiss_mountain_dog
australian_t

## Image Augmentation


In [ ]:
datagen = ImageDataGenerator(
        rotation_range=50,
        width_shift_range=0.3,
        height_shift_range=0.2,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('training_images/afghan_hound/0d5a88f0ab2db8d34b533c69768135e8.jpg') 
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

if not os.path.exists('preview'):
        os.makedirs('preview')

for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='dog_breed', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

## Implementing Convolution Neural Network

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
#check class_mode in keras documentation https://keras.io/preprocessing/image/
training_set = train_datagen.flow_from_directory(
        'training_images',
        target_size=(128, 128),
        batch_size=20,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        'validation_images',
        target_size=(128, 128),
        batch_size=20,
        class_mode='categorical')

In [ ]:
from tensorflow.keras.layers import Dropout
clf = Sequential()
#Convolution
#32 is number of kernals of 3x3, we can use 64 128 256 etc in next layers
#input shape can be 128, 256 later
clf.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#Max Pooling size reduces divided by 2
clf.add(MaxPooling2D(pool_size=(2,2)))      


#clf.add(Dropout(0.5))

clf.add(Conv2D(32,(3,3), activation='relu'))
clf.add(MaxPooling2D(pool_size=(2,2)))
#clf.add(Dropout(0.25))

clf.add(Conv2D(64, (3, 3), activation='relu'))
clf.add(MaxPooling2D(pool_size=(2, 2)))
#clf.add(Dropout(0.10))
#Flattening
clf.add(Flatten())
        
#Adding An ANN
#lets take 128 hidden nodes in hidden layer
#clf.add(Dense(units=128,activation='relu'))
clf.add(Dense(units=64, activation='relu'))
clf.add(Dropout(0.5))
clf.add(Dense(units=120,activation='softmax'))
#stochastic gradient descent -Adam -optimizer
#loss func categorical cross entropy
#metrics = accuracy
clf.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stopping_monitor=EarlyStopping(patience=6)

In [ ]:
hist=clf.fit_generator(
        training_set,
        steps_per_epoch=200,
        epochs=3,
        validation_data=test_set,
        validation_steps=222,
        callbacks=[early_stopping_monitor])

In [ ]:
test_set = []
test_set_ids = []
for curImage in os.listdir('test'):
    test_set_ids.append(os.path.splitext(curImage)[0])
    #print(os.path.splitext(curImage)[0])
    curImage = cv2.imread('test/'+curImage)
    test_set.append(cv2.resize(curImage,(128, 128)))

In [ ]:
test_set = np.array(test_set, np.float32)/255.0

In [ ]:
predictions = clf.predict(test_set)

In [ ]:
predictions[0].shape

In [ ]:
training_set.class_indices

In [ ]:
classes= {index:breed for breed,index in training_set.class_indices.items()}
column_names = [classes[i] for i in range(120)]
column_names

In [ ]:
predictions_df = pd.DataFrame(predictions)
predictions_df.columns = column_names
predictions_df.insert(0,'id', test_set_ids)
#predictions_df.index = test_set_ids
predictions_df

In [ ]:
predictions_df.to_csv('interim_submission.csv',sep=",")

## Graph representation of validation loss vs number of epochs

In [ ]:
plt.plot(hist.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('validation loss')
plt.show()

In [ ]:
plt.plot(hist.history['binary_accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
plt.plot(hist.history['binary_accuracy'],label="Accuracy")
plt.plot(hist.history['val_binary_accuracy'], label="Validation accuracy")
plt.legend()
plt.xlabel('epochs')
plt.show()

In [ ]:
plt.plot(hist.history['loss'],label="traing loss")
plt.plot(hist.history['val_loss'], label="Validation loss")
plt.legend()
plt.xlabel('epochs')
plt.show()

## Observations